In [1]:
import os
os.chdir('../../scripts/external-data-download/')

In [2]:
OUTPUT_RELATIVE_PATH = "../../data/"
def create_path():
    """
        
    """
    path = ["external-raw-data", "curated/external-data", "raw/external-data"]
    for target_dir in path:
        if not os.path.exists(OUTPUT_RELATIVE_PATH + target_dir):
            os.makedirs(OUTPUT_RELATIVE_PATH + target_dir)
    print('Already Create Paths')

create_path()

Already Create Paths


In [3]:
from download import download

download.download_all()

/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/download.py:292: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge_select['Value'] = df_merge_select['Value'].str[1:]
/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/download.py:294: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge_select["Value"] = pd.to_numeric(df_merge_select["Value"])
